### Home Network Assistant
---

In this notebook, we will create a Knowledge base for our Home Network Agent. This agent will have access to a knowledge base for retrieval at runtime which will contain API specs on the home networking system.

Amazon Bedrock Agents enable generative AI applications to execute `multi-step` business tasks using natural language.

In our first example we will create a `Home Networking Assistant` sub agent that will have access to an `API` spec and will be able to generate code to call the `API` based on the user request.

**Agent code generation and execution workflow**:

1. The workflow starts with information retrieval. We create a knowledge base, and store the information from the `Home Network openAPI spec` into the knowledge base. The OpenAPI spec is provided by the user in the `data` folder.

1. This knowledge base will be wrapped within a lambda function that will be invoked based on the user query. It will `retrieve` the top `k` results from the knowledge base and send it as an input to the next step.

1. Next, a `Home Networking` assistant agent will have access to the required API specs to use to generate code. It will use the information from the Knowledge base and generate code for the given API spec, save the code and execute the code based on the parameters provided by the user.

### Setup

Firstly, you are going to install boto3 dependencies from pip. Make sure you have the latest version of it for full capabilities

#### Restart kernel

If you face issues to apply the latest multi-agent capabilities, uncomment this line to restart kernel to ensure packages updates to take effect

In [ ]:
import IPython

# IPython.Application.instance().kernel.do_shutdown(True)

In [ ]:
# Check your boto3 version
!pip freeze | grep boto3

In [ ]:
# import the required packages and libraries
import os
import sys
import boto3
import logging
from typing import Optional
# Get the current file's directory
current_dir = os.path.dirname(os.path.abspath('__file__'))
# Get the parent directory
parent_dir = os.path.dirname(current_dir)
print(parent_dir)
# Add the parent directory to sys.path
sys.path.append(parent_dir)
from globals import *

In [ ]:
# set a logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
# configure the sts client, the boto3 session and other variables
sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = "us-east-1" if session.region_name is None else session.region_name
account_id_suffix = account_id[:3]
agent_suffix = f"{region}-{account_id_suffix}"

s3_client = boto3.client('s3', region)
bedrock_client = boto3.client('bedrock-runtime', region)

### Importing helper functions

On following section, we're adding `knowledge_base_helper.py` on Python path, so the files can be recognized and their functionalities can be invoked. This module contains a helper class for building and using Knowledge Bases for Amazon Bedrock. This class provides a convenient interface for working with KBs such as:

Those files contain helper classes totally focused on make labs experience smoothly.

1. `Creating KBs`: This class contains functions to create, update and invoke knowledge bases
1. `Manage IAM roles`: Contains functions to manage, create and update IAM roles and OpenSearch Serverless.

In this notebook, we specifically use two functions:

1. `create_or_retrieve_knowledge_base`: This function creates a new Knowledge Base or retrieves an existing one.
1. `synchronize_data`: This function synchronizes the Knowledge Base with the storage where data is configured.

In [ ]:
import sys
sys.path.insert(0, ".")
sys.path.insert(1, "..")
# Import utility functions and helper functions for agents
from utils.utils import *
from utils.knowledge_base_helper import (
    KnowledgeBasesForAmazonBedrock
)

# Initialize the KB class
kb = KnowledgeBasesForAmazonBedrock()

### Load the config file
--- 

Load the config file that contains information on the models, data directories, etc.

In [ ]:
# Get the absolute path to the config file
# This config file contains data about the directory paths, the API specs that
# are used to generate the code, and the agent foundation models that are used to generate the code.
BASE_DIR = os.path.abspath(sys.path[1])
CONFIG_FPATH = os.path.join(BASE_DIR, CONFIG_FNAME)
config_data = load_config(CONFIG_FPATH)
logger.info(f"Loaded config from local file system: {json.dumps(config_data, indent=2)}")

### Step 1: Create a knowledge base and wrap it within a lambda function
---

In this portion of the notebook, we will create a knowledge base. This knowledge base is for the home networking API spec information and will be wrapped within a lambda function. It will use the `retrieve` API to return the top `k` results based on the user query. 

Once we have created the knowledge base, we will wrap it in a lambda function and test it. As a part of this test, the user will be able to ask any question about the API spec and retrieve the top `k` results based on the query.

**Note**: We are using `HYBRID` search in this retrieval. Hybrid search brings the best of both approaches: precision of semantic search and coverage of keywords. It works great for RAG-based applications where the retriever has to handle a wide variety of natural language queries.

In [ ]:
# This is the bucket where the API spec is loaded and synced from into the KB. If this bucket is not 
# configured, it is automatically created.
HOME_NETWORK_S3_BUCKET: str = config_data['dir_paths']['knowledge_base_info'].get('home_network_knowledge_bucket')
logger.info(f"Home Network S3 bucket configured by the user: {HOME_NETWORK_S3_BUCKET}")

In [ ]:
%%time
# First ensure the bucket exists. This bucket will contain information about the spec
# which will be synced with the knowledge base. In this case, we are creating an "home-network-kb" bucket
# to store information about the home network API spec
if create_s3_bucket_for_kb(HOME_NETWORK_S3_BUCKET, region):
    home_network_kb_id, ds_id = kb.create_or_retrieve_knowledge_base(
        HOME_NETWORK_KB_NAME,
        HOME_NETWORK_KB_DESCRIPTION,
        HOME_NETWORK_S3_BUCKET, 
    )
    logger.info(f"Knowledge Base ID: {home_network_kb_id}")
    logger.info(f"Data Source ID: {ds_id}")
else:
    logger.error("Failed to create or verify S3 bucket. Cannot proceed with knowledge base creation.")

### Load the Home Networking data in the `Home Network Knowledge Base`
---

In this portion of the notebook, we will load the Home Network API spec as a `JSON` file in the configured S3 bucket. After that, we will sync the `KB` with data in S3 and use that as a tool for our Home networking agent to look for the relevant API and generate code to call the API.

In [ ]:
# retrieve the FMC spec json file from the data directory
home_network_api_sec_fpath: str = os.path.join(BASE_DIR, config_data['dir_paths']['data_prefix'], config_data['dir_paths']['api_specs'].get('home_network_api_spec'))
logger.info(f"Uploading the Home networking API spec '{home_network_api_sec_fpath}' to {HOME_NETWORK_S3_BUCKET}")
upload_file_to_s3(home_network_api_sec_fpath, HOME_NETWORK_S3_BUCKET)
logger.info(f"Successfully uploaded the Home networking API spec to {HOME_NETWORK_S3_BUCKET}")

### Synchronize the Knowledge Base with Data in `S3`
---

Now we will sync the data from the S3 bucket into the Home Networking Knowledge Base

In [ ]:
# sync knowledge base
kb.synchronize_data(home_network_kb_id, ds_id)

In [ ]:
kb_info = kb.get_kb(home_network_kb_id)
kb_arn = kb_info['knowledgeBase']['knowledgeBaseArn']

# Initialize the kb config that we will pass to the sub agent
kb_config = {
    'kb_id': home_network_kb_id,
    'kb_instruction': """Access this knowledge base to look for the correct API based on the user query. Retrieve the relevant searches based on the user query."""
}

### Test the results from the `Home Networking knowledge base`
---

In this portion of the step, we will test the Home Networking knowledge base with the `retrieve API` to retrieve the top `k` relevant searches based on the user query.

In [ ]:
KNOWLEDGE_BASE_INFO: str = config_data['dir_paths']['knowledge_base_info']
logger.info(f"Provided knowledge base information: {json.dumps(KNOWLEDGE_BASE_INFO, indent=2)}")

In [ ]:
%%time
query = """Is my front door camera online right now?"""
resp = query_knowledge_base(query, home_network_kb_id, KNOWLEDGE_BASE_INFO)

In [ ]:
logger.info(f"Response from the KB {home_network_kb_id}: {json.dumps(resp, indent=2)}")

### Wrap the knowledge base in a lambda function
---

Next, we will wrap the function above in a lambda function and then invoke the lambda function to get the responses

In [ ]:
# Deploy the Lambda
function_arn = create_kb_lambda(
    lambda_function_name=HOME_NETWORK_KB_LAMBDA_FUNCTION_NAME,
    source_code_file=HOME_NETWORK_KB_LAMBDA_FUNCTION,
    region=region,
    kb_id=home_network_kb_id
)

logger.info(f"Lambda function ARN: {function_arn}")

In [ ]:
time.sleep(30)

In [ ]:
%%time
query = """How much storage is left on my garage camera?"""
resp = query_lambda(query, region, home_network_kb_id, HOME_NETWORK_KB_LAMBDA_FUNCTION_NAME)

# Print results
print("\nRetrieved chunks:")
for chunk in resp['chunks']:
    print(f"\nText: {chunk['text']}")
    print(f"Score: {chunk['score']}")
    print("-" * 50)

In [ ]:
resp

In [ ]:
%%time

# This points to the signalStrength field in the CameraStatus
query = """What is the signal strength of my porch camera?"""
resp = query_lambda(query, region, home_network_kb_id, HOME_NETWORK_KB_LAMBDA_FUNCTION_NAME)

# Print results
print("\nRetrieved chunks:")
for chunk in resp['chunks']:
    print(f"\nText: {chunk['text']}")
    print(f"Score: {chunk['score']}")
    print("-" * 50)

In [ ]:
resp

In [ ]:
home_network_kb_id

In [ ]:
%store home_network_kb_id